# Сетки для подбора параметров

In [1]:
import random
import time
import cellbell

import pickle
import gc

import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import inv
from scipy.linalg import sqrtm
from scipy.sparse import eye
from scipy import linalg
import scipy
from scipy import stats

from cvxpy import *

#import Pre_processing

## Эти функции потом портить через модуль!!

In [2]:
def inv_A_count_and_record( C_r, X, file_name_for_record = False ):
    """
    Возвращает посчитанную инвертированную матрицу А. 
    В качестве Х нужно матрицу нативных признаков.
    Вернет симметричную квадратную матрицу размера n*n, где n - количество признаков
    """
    X_T_X = (X.T @ X).A
    size = (X_T_X).shape[0]
    
    values, vectors = np.linalg.eigh( ( np.identity( size ) + (2*C_r)*X_T_X ) )
    sqrt_A = vectors @ np.diag(np.sqrt(values)) @ np.linalg.inv(vectors)
    
    #---------------Другой способ-------------------------------
    
    #inv_A_1 = sqrtm( ( scipy.sparse.eye( size ) + 2*C_r*X_T_X ).A)
    
    #------------------------------------------------------------
    
    sqrt_A = 0.5*(sqrt_A + sqrt_A.T)
    
    inv_A = np.linalg.inv(sqrt_A)
    
    del sqrt_A
    
    inv_A = 0.5*(inv_A + inv_A.T)
    
    del X_T_X
    
    ####################
    inv_A = np.where( (inv_A < 1e-9) & (inv_A > 0), 0, inv_A)
    ####################
    
    if file_name_for_record == False:
        return inv_A
    else:
        np.save( inv_A_csr, file_name_for_record )

In [3]:
def d_count_and_record( C_r, X, inv_A, s, file_name_for_record = False ):
    """
    В качестве X следует подать матрицу с комплексами в нативной конформации
    Вернет вектор, длиной n, где n - количество признаков
    """
    X_T_s = X.T.A @ s
    d = (2 * C_r) * (inv_A @ X_T_s)
    
    del X_T_s
    
    if file_name_for_record == False:
        return d
    else: 
        np.save(file_name_for_record, d)

In [4]:
def new_X_count_and_record( inv_A, X, file_name_for_record =False ):
    """
    В качестве X следует подать полную матрицу положений, а в качестве inv_A обращенную матрицу A посчитанную на нативных положениях
    Произведет замену и вернет новую матрицу признаков
    """
    #conv_X = inv_A * X.T
    conv_X = X.A @ inv_A
    if file_name_for_record == False:
        return conv_X
    else:
        CSR_write(conv_X, file_name_for_record)

In [5]:
def creat_and_record_vector_ydx( X, y, d, file_name_for_record = False ):
    
    ydx = []
    for ind, value in enumerate(y):
            ydx.append( (value * X[ind]) @ d -1.0 )
    
    if file_name_for_record == False:
        return np.array(ydx)
    else:
        np.save(file_name_for_record, ydx)

In [6]:
def index_with_native_conformation(y, file_name_for_record = False):
    """
    Нужна для того, чтобы не нужно было считать скалярные произведения там, где значене будет 0 из-за y_i
    """
    not_null_index = []
    for index, value in enumerate(y):
        if value == 1:
            not_null_index.append(index)
            
    if file_name_for_record == False:    
        return not_null_index
    else:
        np.save(file_name_for_record, not_null_index)

In [7]:
def w_count_and_record(lambda_v, y, X, inv_A, d):
            
    ly = lambda_v * y
    Xly = X.A.T @ ly
    w_transformed = inv_A @ Xly
    
    del Xly
    del ly
    
    w_t_d = w_transformed + d
    w = inv_A @ w_t_d
    
    del w_t_d
    
    return w

---

In [76]:
! free -m

              total        used        free      shared  buff/cache   available
Mem:          64311        2025        2043         139       60242       61567
Swap:             0           0           0


## Загрузим необходимые данные

In [19]:
y_1000_float64 = np.load("Data/1000_64float_good/labels_general_t14_t3_l7.0_g1.0_r1.0sasa_solvent_all_float64_first1000.npy").astype(dtype=np.float)
s_1000_float64 = np.load("Data/1000_64float_good/affinities_general_t14_t3_l7.0_g1.0_r1.0sasa_solvent_all_float64_first1000.npy")

loader = np.load("Data/1000_64float_good/data_array_general_t14_t3_l7.0_g1.0_r1.0sasa_solvent_all_float64_first1000_sparse.npz")
X_1000_float64 = csr_matrix((loader['data'], loader['indices'], loader['indptr']),
               shape=loader['shape']) 
del loader

---

In [160]:
obj = (y_1000_float64 * (X_1000_float64_new.T)) @ lambda_

In [161]:
obj

Expression(AFFINE, UNKNOWN, (6819,))

In [165]:
g = cvxpy.power(np.array([1,2,3]), 2)

In [166]:
g.value

array([1., 4., 9.])

In [170]:
h = cvxpy.multiply(np.array([1,2,3]), np.array([1,2,3]))

In [171]:
h.value

array([1., 4., 9.])

## Сетки

### Сетка 1: Сетка с всеми признаками от классификации и с flexibility
Признаки никак не обрабатывались. Ошибка в вычислениях b не была исправлена

$C_r$ in [10, 1, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001]

$C$ in [0.1, 0.5, 1, 5, 10, 15]

In [17]:
C = [0.1, 0.5, 1, 5, 10, 15]
C_r = [10, 1, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001]
Grid_name = "output_grids/Grid_1/Grid_1_"

In [99]:
ind = [i for i in range(6440)]
ind.append(6818)

X_1000_float64_1 = X_1000_float64[:,ind]

X_1000_float64_1.shape

(19000, 6441)

In [66]:
P = 1000
D = 19

set_i_and_j = range(1, P*D+1)
set_k = range(1, D+1)
set_l = range(0,P*D,D)

In [20]:
for c in C:
    for c_r in C_r:
        #----------------------------------------------------------------------------
        
        index_native_conf = index_with_native_conformation(y_1000_float64)
        
        X_1000_float64_native = X_1000_float64_1[index_native_conf]
        
        inv_A = inv_A_count_and_record( c_r, X_1000_float64_native )
        
        d = d_count_and_record( c_r, X_1000_float64_native, inv_A, s_1000_float64 )
        
        X_1000_float64_new = new_X_count_and_record( inv_A, X_1000_float64_1 )
        
        ydx = creat_and_record_vector_ydx( X_1000_float64_new, y_1000_float64, d )
        
        #-----------------------------------------------------------------------------

        lambda_ = cvxpy.Variable(P*D)

        constraints = [0 <= lambda_, lambda_ <= c]
        constraints += [ lambda_[i:i+D].T @ y_1000_float64[i:i+D] == 0 for i in set_l]
        
        objective = cvxpy.Minimize(0.5*cvxpy.sum_squares( (y_1000_float64 * (X_1000_float64_new.T)) @ lambda_) + ydx.T @ lambda_)
        
        prob = cvxpy.Problem(objective,constraints)
        
        prob.solve(solver=cvxpy.OSQP, verbose=True, max_iter=40000)
        
        #-----------------------------------------------------------------------------

        lambda_v = lambda_.value
        
        #Забыл, что надо записывать w, поэтому на первой итерации у меня записаны lambda
        w = w_count_and_record(lambda_v, y_1000_float64, X_1000_float64_1, inv_A, d)
        
        #-----------------------------------------------------------------------------
        
        np.save(Grid_name+'C_'+str(c)+'_Cr_'+str(c_r),lambda_v)
        
        print("Now count C = {0}, C_r = {1}\n".format(c,c_r))

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 120738882
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 40000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -3.6166e+03   1.47e+00   6.46e+00   1.00e-01   5.06e+02s
 200  -2.4729e+02   1.62e-04   2.47e-03   1.00e-01   5.74e+02s
 375  -2.4730e+02   2.53e-06   1.08

   1  -3.0858e+03   1.51e+00   1.25e+01   1.00e-01   5.02e+02s
 200  -2.0059e+02   1.68e-04   5.94e-04   1.00e-01   5.70e+02s
 400  -2.0059e+02   7.61e-05   2.58e-04   1.00e-01   6.36e+02s
 600  -2.0059e+02   2.35e-05   1.63e-04   1.00e-01   7.01e+02s
 800  -2.0059e+02   2.90e-05   1.42e-04   1.00e-01   7.67e+02s
 925  -2.0059e+02   8.83e-06   1.38e-04   1.00e-01   8.08e+02s

status:               solved
solution polish:      unsuccessful
number of iterations: 925
optimal objective:    -200.5895
run time:             1.35e+03s
optimal rho estimate: 2.41e-01

Now count C = 0.1, C_r = 0.001

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 120738882
settings: lin

Now count C = 0.5, C_r = 0.1

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 120738882
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 40000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -2.3875e+03   6.72e-01   8.16e+00   1.00e-01   5.03e+02s
 200  -1.1066e+03   6.93e-04   2.58e-03   1.00e-01   5.68e+02s
 400 

 200  -9.9993e+02   4.59e-04   1.52e-03   1.00e-01   5.67e+02s
 400  -9.9997e+02   1.25e-04   9.08e-04   1.00e-01   6.33e+02s
 600  -9.9998e+02   1.16e-04   8.18e-04   1.00e-01   6.98e+02s
 800  -9.9999e+02   9.72e-05   8.00e-04   1.00e-01   7.63e+02s
1000  -9.9999e+02   1.34e-04   5.66e-04   1.00e-01   8.29e+02s
1200  -1.0000e+03   7.13e-05   5.52e-04   1.00e-01   8.94e+02s
1400  -1.0000e+03   4.71e-05   4.58e-04   1.00e-01   9.59e+02s
1600  -1.0000e+03   7.69e-05   4.20e-04   1.00e-01   1.02e+03s
1800  -1.0000e+03   5.57e-05   3.21e-04   1.00e-01   1.09e+03s
2000  -1.0000e+03   2.44e-05   2.10e-04   1.00e-01   1.16e+03s
2025  -1.0000e+03   1.15e-05   2.09e-04   1.00e-01   1.17e+03s

status:               solved
solution polish:      unsuccessful
number of iterations: 2025
optimal objective:    -1000.0010
run time:             1.69e+03s
optimal rho estimate: 7.01e-02

Now count C = 0.5, C_r = 0.0005

-----------------------------------------------------------------
           OSQP v0.

   1  -2.3875e+03   4.08e-01   8.82e+00   1.00e-01   5.07e+02s
 200  -2.1662e+03   2.21e-03   5.07e-03   1.00e-01   5.75e+02s
 400  -2.1663e+03   6.72e-04   2.20e-03   1.00e-01   6.40e+02s
 600  -2.1663e+03   1.90e-04   1.73e-03   1.00e-01   7.05e+02s
 800  -2.1663e+03   2.96e-05   9.35e-04   1.00e-01   7.71e+02s
1000  -2.1663e+03   4.58e-05   7.67e-04   1.00e-01   8.37e+02s
1200  -2.1663e+03   3.12e-05   5.44e-04   1.00e-01   9.02e+02s
1400  -2.1663e+03   4.19e-07   5.18e-04   1.00e-01   9.68e+02s
1600  -2.1663e+03   3.69e-07   4.90e-04   1.00e-01   1.03e+03s
1800  -2.1663e+03   4.91e-06   3.43e-04   1.00e-01   1.10e+03s
2000  -2.1663e+03   8.85e-07   3.23e-04   1.00e-01   1.16e+03s
2200  -2.1663e+03   9.76e-06   2.78e-04   1.00e-01   1.23e+03s
2400  -2.1663e+03   2.44e-07   1.95e-04   1.00e-01   1.30e+03s
2600  -2.1663e+03   2.43e-06   1.37e-04   1.29e-02   1.84e+03s

status:               solved
solution polish:      unsuccessful
number of iterations: 2600
optimal objective:    -216

...

Now count C = 10, C_r = 0.05

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 120738882
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 40000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -2.2851e+03   1.42e-01   1.08e+01   1.00e-01   5.01e+02s
 200  -1.9930e+04   7.86e-03   3.39e-02   1.00e-01   5.67e+02s
 400 

 400  -1.9924e+04   4.72e-03   1.72e-02   1.00e-01   6.36e+02s
 600  -1.9930e+04   2.93e-03   1.58e-02   1.00e-01   7.01e+02s
 800  -1.9934e+04   1.65e-03   1.26e-02   1.00e-01   7.68e+02s
1000  -1.9936e+04   1.95e-03   8.85e-03   1.00e-01   8.34e+02s
1200  -1.9938e+04   2.32e-03   8.90e-03   1.00e-01   9.00e+02s
1400  -1.9939e+04   7.18e-04   7.02e-03   1.00e-01   9.65e+02s
1600  -1.9939e+04   1.00e-03   5.68e-03   1.00e-01   1.03e+03s
1800  -1.9940e+04   1.02e-03   5.25e-03   1.00e-01   1.10e+03s
2000  -1.9940e+04   2.24e-04   4.21e-03   1.00e-01   1.16e+03s
2200  -1.9941e+04   2.46e-04   3.79e-03   1.00e-01   1.23e+03s
2400  -1.9941e+04   3.06e-04   3.46e-03   1.00e-01   1.29e+03s
2600  -1.9941e+04   4.14e-04   3.53e-03   1.00e-01   1.36e+03s
2800  -1.9941e+04   1.20e-04   3.57e-03   1.00e-01   1.43e+03s
3000  -1.9942e+04   8.19e-05   3.56e-03   1.00e-01   1.49e+03s
3200  -1.9942e+04   5.63e-03   1.30e-03   1.14e-02   2.04e+03s
3400  -1.9943e+04   1.56e-03   8.69e-04   1.14e-02   2.


status:               solved
solution polish:      unsuccessful
number of iterations: 2525
optimal objective:    -33663.1296
run time:             2.35e+03s
optimal rho estimate: 9.38e-04

Now count C = 15, C_r = 10

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 120738882
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 40000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

i

4600  -3.0163e+04   6.93e-04   1.06e-03   1.55e-02   2.50e+03s
4800  -3.0163e+04   5.29e-04   9.88e-04   1.55e-02   2.57e+03s
5000  -3.0163e+04   2.53e-04   9.54e-04   1.55e-02   2.63e+03s
5200  -3.0163e+04   5.95e-05   8.64e-04   1.55e-02   2.70e+03s
5400  -3.0163e+04   9.41e-04   8.09e-04   1.55e-02   2.76e+03s
5600  -3.0163e+04   1.92e-05   7.38e-04   1.55e-02   2.83e+03s
5800  -3.0163e+04   1.02e-04   7.11e-04   1.55e-02   2.89e+03s
6000  -3.0163e+04   8.94e-04   6.25e-04   1.55e-02   2.96e+03s
6200  -3.0163e+04   3.24e-05   6.26e-04   1.55e-02   3.02e+03s
6400  -3.0163e+04   1.57e-05   6.21e-04   1.55e-02   3.09e+03s
6600  -3.0163e+04   5.31e-05   6.23e-04   1.55e-02   3.16e+03s
6800  -3.0163e+04   1.84e-06   6.22e-04   1.55e-02   3.22e+03s
7000  -3.0163e+04   9.89e-06   6.15e-04   1.55e-02   3.29e+03s
7200  -3.0163e+04   2.48e-05   6.07e-04   1.55e-02   3.35e+03s
7400  -3.0163e+04   3.39e-04   4.06e-04   2.94e-03   3.90e+03s
7600  -3.0163e+04   2.26e-05   1.41e-04   2.94e-03   3.

 200  -2.9792e+04   1.64e-02   5.00e-02   1.00e-01   5.70e+02s
 400  -2.9828e+04   7.84e-03   2.59e-02   1.00e-01   6.35e+02s
 600  -2.9843e+04   4.19e-03   2.36e-02   1.00e-01   7.01e+02s
 800  -2.9851e+04   4.90e-03   1.72e-02   1.00e-01   7.66e+02s
1000  -2.9856e+04   1.77e-03   1.41e-02   1.00e-01   8.32e+02s
1200  -2.9859e+04   1.12e-03   1.33e-02   1.00e-01   8.98e+02s
1400  -2.9866e+04   6.64e-03   5.84e-03   1.89e-02   1.44e+03s
1600  -2.9868e+04   7.64e-03   4.51e-03   1.89e-02   1.51e+03s
1800  -2.9869e+04   5.62e-03   2.96e-03   1.89e-02   1.57e+03s
2000  -2.9870e+04   1.25e-03   1.80e-03   1.89e-02   1.64e+03s
2200  -2.9870e+04   8.12e-04   1.32e-03   1.89e-02   1.71e+03s
2400  -2.9870e+04   2.02e-03   1.14e-03   1.89e-02   1.77e+03s
2600  -2.9871e+04   1.99e-03   1.06e-03   1.89e-02   1.84e+03s
2800  -2.9871e+04   1.57e-04   9.73e-04   1.89e-02   1.90e+03s
3000  -2.9871e+04   1.74e-04   8.93e-04   1.89e-02   1.97e+03s
3200  -2.9871e+04   2.90e-04   7.85e-04   1.89e-02   2.

### Сетка 2: Сетка с всеми признаками от классификации и с flexibility
Признаки никак не обрабатывались. Ошибка в вычислениях d исправлена

$C_r$ in [10, 1, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001, 0]

$C$ in [0.1, 0.5, 1, 5, 10, 15]

In [60]:
C_r = [10, 1, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001, 0]
C = [0.1, 0.5, 1, 5, 10, 15]
Grid_name = "output_grids/Grid_2/Grid_2_"

In [61]:
ind = [i for i in range(6440)]
ind.append(6818)

X_1000_float64_1 = X_1000_float64[:,ind]

X_1000_float64_1.shape

(19000, 6441)

In [62]:
P = 1000
D = 19

set_i_and_j = range(1, P*D+1)
set_k = range(1, D+1)
set_l = range(0,P*D,D)

In [63]:
for c in C:
    for c_r in C_r:
        #----------------------------------------------------------------------------
        
        index_native_conf = index_with_native_conformation(y_1000_float64)
        
        X_1000_float64_native = X_1000_float64_1[index_native_conf]
        
        inv_A = inv_A_count_and_record( c_r, X_1000_float64_native )
        
        d = d_count_and_record( c_r, X_1000_float64_native, inv_A, s_1000_float64 )
        
        X_1000_float64_new = new_X_count_and_record( inv_A, X_1000_float64_1 )
        
        ydx = creat_and_record_vector_ydx( X_1000_float64_new, y_1000_float64, d )
        
        #-----------------------------------------------------------------------------

        lambda_ = cvxpy.Variable(P*D)

        constraints = [0 <= lambda_, lambda_ <= c]
        constraints += [ lambda_[i:i+D].T @ y_1000_float64[i:i+D] == 0 for i in set_l]
        
        objective = cvxpy.Minimize(0.5*cvxpy.sum_squares( (y_1000_float64 * (X_1000_float64_new.T)) @ lambda_ ) + ydx.T @ lambda_)
        
        prob = cvxpy.Problem(objective,constraints)
        
        prob.solve(solver=cvxpy.OSQP, verbose=True, max_iter=40000)
        
        #-----------------------------------------------------------------------------

        lambda_v = lambda_.value
        
        w = w_count_and_record(lambda_v, y_1000_float64, X_1000_float64_1, inv_A, d)
        
        #-----------------------------------------------------------------------------
        
        np.save(Grid_name+'C_'+str(c)+'_Cr_'+str(c_r),w)
        
        print("Now count C = {0}, C_r = {1}\n".format(c,c_r))

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 120738882
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 40000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -3.3077e+03   1.34e+00   8.79e+00   1.00e-01   5.39e+02s
 200  -2.6701e+02   4.47e-04   2.16e-03   1.00e-01   6.07e+02s
 400  -2.6702e+02   1.43e-05   2.63

   1  -2.2375e+03   1.06e+00   1.73e+01   1.00e-01   5.08e+02s
 200  -2.0114e+02   2.54e-04   6.47e-04   1.00e-01   5.77e+02s
 400  -2.0115e+02   9.71e-05   3.61e-04   1.00e-01   6.44e+02s
 600  -2.0115e+02   4.00e-05   3.30e-04   1.00e-01   7.11e+02s
 800  -2.0115e+02   3.38e-05   2.99e-04   1.00e-01   7.77e+02s
 900  -2.0115e+02   5.59e-06   1.63e-04   1.00e-01   8.11e+02s

status:               solved
solution polish:      unsuccessful
number of iterations: 900
optimal objective:    -201.1483
run time:             1.34e+03s
optimal rho estimate: 1.71e-01

Now count C = 0.1, C_r = 0.001

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 120738882
settings: lin

   1  -1.8709e+03   5.14e-01   1.12e+01   1.00e-01   5.05e+02s
 200  -1.1642e+03   1.78e-03   5.36e-03   1.00e-01   5.73e+02s
 400  -1.1642e+03   5.82e-05   1.40e-03   1.00e-01   6.45e+02s
 600  -1.1642e+03   9.57e-07   8.49e-04   1.00e-01   7.13e+02s
 800  -1.1642e+03   5.57e-07   6.97e-04   1.00e-01   7.79e+02s
1000  -1.1642e+03   3.37e-07   5.53e-04   1.00e-01   8.48e+02s
1200  -1.1642e+03   7.91e-06   1.83e-04   1.42e-02   1.40e+03s

status:               solved
solution polish:      unsuccessful
number of iterations: 1200
optimal objective:    -1164.2402
run time:             1.92e+03s
optimal rho estimate: 1.78e-02

Now count C = 0.5, C_r = 0.05

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constrain

   1  -3.3793e+03   1.50e+00   1.86e+01   1.00e-01   5.08e+02s
 200  -9.9978e+02   3.36e-04   1.89e-03   1.00e-01   5.76e+02s
 400  -9.9982e+02   2.49e-04   1.44e-03   1.00e-01   6.43e+02s
 600  -9.9984e+02   2.03e-04   1.07e-03   1.00e-01   7.10e+02s
 800  -9.9985e+02   1.54e-04   8.35e-04   1.00e-01   7.77e+02s
1000  -9.9985e+02   6.38e-05   8.29e-04   1.00e-01   8.44e+02s
1200  -9.9985e+02   1.24e-04   8.40e-04   1.00e-01   9.10e+02s
1400  -9.9986e+02   7.26e-05   8.28e-04   1.00e-01   9.77e+02s
1600  -9.9986e+02   4.95e-05   8.11e-04   1.00e-01   1.04e+03s
1800  -9.9986e+02   2.44e-05   7.03e-04   1.00e-01   1.11e+03s
2000  -9.9986e+02   4.32e-05   5.75e-04   1.00e-01   1.18e+03s
2200  -9.9986e+02   2.34e-05   5.55e-04   1.00e-01   1.24e+03s
2400  -9.9986e+02   2.22e-05   5.10e-04   1.00e-01   1.31e+03s
2600  -9.9986e+02   7.60e-06   4.99e-04   1.00e-01   1.38e+03s
2800  -9.9986e+02   3.79e-05   4.83e-04   1.00e-01   1.44e+03s
3000  -9.9986e+02   2.56e-05   3.68e-04   1.00e-01   1.

   1  -1.6856e+03   1.96e-01   1.43e+01   1.00e-01   5.09e+02s
 200  -2.0651e+03   1.52e-03   7.36e-03   1.00e-01   5.77e+02s
 400  -2.0655e+03   6.39e-04   3.86e-03   1.00e-01   6.48e+02s
 600  -2.0657e+03   9.39e-04   2.11e-03   1.00e-01   7.16e+02s
 800  -2.0657e+03   2.25e-04   1.65e-03   1.00e-01   7.82e+02s
1000  -2.0657e+03   2.61e-04   1.35e-03   1.00e-01   8.49e+02s
1200  -2.0658e+03   1.74e-04   9.96e-04   1.00e-01   9.16e+02s
1400  -2.0658e+03   1.53e-05   9.77e-04   1.00e-01   9.83e+02s
1600  -2.0658e+03   1.34e-04   7.69e-04   1.00e-01   1.05e+03s
1800  -2.0658e+03   4.10e-05   6.40e-04   1.00e-01   1.12e+03s
2000  -2.0658e+03   2.02e-05   5.75e-04   1.00e-01   1.18e+03s
2200  -2.0658e+03   1.41e-05   5.26e-04   1.00e-01   1.25e+03s
2400  -2.0658e+03   2.60e-05   4.96e-04   1.00e-01   1.32e+03s
2600  -2.0658e+03   1.59e-05   3.93e-04   1.00e-01   1.38e+03s
2800  -2.0658e+03   2.04e-06   3.61e-04   1.00e-01   1.45e+03s
3000  -2.0658e+03   1.46e-04   2.80e-04   1.20e-02   2.

...

7600  -2.0009e+04   8.40e-05   4.23e-04   1.49e-02   3.54e+03s
7800  -2.0009e+04   1.55e-05   4.09e-04   1.49e-02   3.61e+03s
8000  -2.0009e+04   2.19e-04   4.04e-04   1.49e-02   3.67e+03s
8200  -2.0009e+04   1.67e-05   3.96e-04   1.49e-02   3.74e+03s
8400  -2.0009e+04   2.65e-05   3.13e-04   1.49e-02   3.81e+03s
8600  -2.0009e+04   4.00e-06   2.89e-04   1.49e-02   3.87e+03s
8800  -2.0009e+04   9.44e-06   2.77e-04   1.49e-02   3.94e+03s
9000  -2.0009e+04   2.20e-04   2.65e-04   1.49e-02   4.01e+03s
9200  -2.0009e+04   7.97e-06   2.52e-04   1.49e-02   4.08e+03s
9400  -2.0009e+04   2.92e-06   2.37e-04   1.49e-02   4.14e+03s
9525  -2.0009e+04   1.03e-06   2.28e-04   1.49e-02   4.18e+03s

status:               solved
solution polish:      unsuccessful
number of iterations: 9525
optimal objective:    -20009.1064
run time:             4.70e+03s
optimal rho estimate: 1.24e-03

Now count C = 10, C_r = 0.01

-----------------------------------------------------------------
           OSQP v0.6.

 200  -1.9902e+04   1.19e-02   3.96e-02   1.00e-01   5.83e+02s
 400  -1.9920e+04   4.91e-03   2.69e-02   1.00e-01   6.50e+02s
 600  -1.9927e+04   1.54e-03   1.77e-02   1.00e-01   7.17e+02s
 800  -1.9931e+04   1.68e-03   1.62e-02   1.00e-01   7.84e+02s
1000  -1.9934e+04   1.83e-03   1.57e-02   1.00e-01   8.51e+02s
1200  -1.9936e+04   1.59e-03   1.52e-02   1.00e-01   9.18e+02s
1400  -1.9939e+04   1.03e-02   8.53e-03   1.87e-02   1.47e+03s
1600  -1.9941e+04   3.04e-03   4.76e-03   1.87e-02   1.54e+03s
1800  -1.9942e+04   1.97e-03   3.59e-03   1.87e-02   1.61e+03s
2000  -1.9942e+04   9.00e-04   2.84e-03   1.87e-02   1.67e+03s
2200  -1.9942e+04   2.72e-03   2.14e-03   1.87e-02   1.74e+03s
2400  -1.9942e+04   1.16e-03   1.28e-03   1.87e-02   1.81e+03s
2600  -1.9942e+04   3.24e-04   8.95e-04   1.87e-02   1.87e+03s
2800  -1.9942e+04   1.24e-03   8.56e-04   1.87e-02   1.94e+03s
3000  -1.9942e+04   1.01e-04   7.59e-04   1.87e-02   2.01e+03s
3200  -1.9943e+04   1.06e-04   7.38e-04   1.87e-02   2.

3800  -3.3557e+04   8.75e-05   2.62e-04   1.12e-03   2.77e+03s
3925  -3.3557e+04   3.49e-05   5.46e-05   1.12e-03   2.81e+03s

status:               solved
solution polish:      unsuccessful
number of iterations: 3925
optimal objective:    -33557.0375
run time:             3.34e+03s
optimal rho estimate: 9.29e-04

Now count C = 15, C_r = 1

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 120738882
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 40000
          check_terminati

   1  -1.6856e+03   1.96e-01   1.43e+01   1.00e-01   5.16e+02s
 200  -2.9805e+04   1.12e-02   6.56e-02   1.00e-01   5.83e+02s
 400  -2.9859e+04   5.03e-03   4.54e-02   1.00e-01   6.55e+02s
 600  -2.9882e+04   5.40e-03   2.94e-02   1.00e-01   7.23e+02s
 800  -2.9894e+04   3.47e-03   2.00e-02   1.00e-01   7.90e+02s
1000  -2.9903e+04   2.25e-03   1.68e-02   1.00e-01   8.57e+02s
1200  -2.9909e+04   3.20e-03   1.41e-02   1.00e-01   9.24e+02s
1400  -2.9913e+04   1.74e-03   1.28e-02   1.00e-01   9.90e+02s
1600  -2.9916e+04   6.06e-04   1.19e-02   1.00e-01   1.06e+03s
1800  -2.9919e+04   1.21e-03   1.01e-02   1.00e-01   1.12e+03s
2000  -2.9921e+04   4.16e-04   8.72e-03   1.00e-01   1.19e+03s
2200  -2.9922e+04   4.24e-04   7.85e-03   1.00e-01   1.26e+03s
2400  -2.9924e+04   6.16e-04   7.23e-03   1.00e-01   1.33e+03s
2600  -2.9925e+04   9.14e-04   6.84e-03   1.00e-01   1.39e+03s
2800  -2.9926e+04   3.59e-04   6.82e-03   1.00e-01   1.46e+03s
3000  -2.9927e+04   1.45e-03   5.35e-03   1.00e-01   1.

2800  -2.9870e+04   3.73e-04   1.78e-03   1.44e-02   2.00e+03s
3000  -2.9871e+04   8.93e-04   1.28e-03   1.44e-02   2.07e+03s
3200  -2.9871e+04   2.66e-04   1.10e-03   1.44e-02   2.14e+03s
3400  -2.9871e+04   1.63e-03   1.05e-03   1.44e-02   2.20e+03s
3600  -2.9871e+04   2.10e-04   9.58e-04   1.44e-02   2.27e+03s
3800  -2.9871e+04   1.91e-04   8.73e-04   1.44e-02   2.34e+03s
4000  -2.9871e+04   3.46e-04   7.78e-04   1.44e-02   2.40e+03s
4200  -2.9871e+04   7.03e-05   7.26e-04   1.44e-02   2.47e+03s
4400  -2.9871e+04   5.86e-05   6.33e-04   1.44e-02   2.54e+03s
4600  -2.9871e+04   2.16e-04   6.79e-04   1.44e-02   2.61e+03s
4800  -2.9871e+04   1.26e-05   7.14e-04   1.44e-02   2.67e+03s
5000  -2.9871e+04   2.05e-05   7.17e-04   1.44e-02   2.74e+03s
5200  -2.9871e+04   3.01e-04   7.26e-04   1.44e-02   2.81e+03s
5400  -2.9871e+04   5.11e-05   7.08e-04   1.44e-02   2.87e+03s
5600  -2.9871e+04   1.80e-06   6.93e-04   1.44e-02   2.94e+03s
5800  -2.9871e+04   2.21e-06   6.78e-04   1.44e-02   3.

---

### Сетка 3: Сетка с всеми признаками от классификации и с flexibility
Признаки никак не обрабатывались. Ошибка в вычислениях d исправлена

$C_r$ in [10]

$C$ in [15,25,50,100,150,200,250,300]

In [116]:
C_r = [10]
C = [15,25,50,100,150,200,250,300]
Grid_name = "output_grids/Grid_3/Grid_3_"

In [117]:
ind = [i for i in range(6440)]
ind.append(6818)

X_1000_float64_1 = X_1000_float64[:,ind]

X_1000_float64_1.shape

(19000, 6441)

In [118]:
P = 1000
D = 19

set_i_and_j = range(1, P*D+1)
set_k = range(1, D+1)
set_l = range(0,P*D,D)

In [122]:
for c in C:
    for c_r in C_r:
        #----------------------------------------------------------------------------
        
        index_native_conf = index_with_native_conformation(y_1000_float64)
        
        X_1000_float64_native = X_1000_float64_1[index_native_conf]
        
        inv_A = inv_A_count_and_record( c_r, X_1000_float64_native )
        
        d = d_count_and_record( c_r, X_1000_float64_native, inv_A, s_1000_float64 )
        
        X_1000_float64_new = new_X_count_and_record( inv_A, X_1000_float64_1 )
        
        ydx = creat_and_record_vector_ydx( X_1000_float64_new, y_1000_float64, d )
        
        #-----------------------------------------------------------------------------

        lambda_ = cvxpy.Variable(P*D)

        constraints = [0 <= lambda_, lambda_ <= c]
        constraints += [ lambda_[i:i+D].T @ y_1000_float64[i:i+D] == 0 for i in set_l]
        
        objective = cvxpy.Minimize(0.5*cvxpy.sum_squares( (y_1000_float64 * (X_1000_float64_new.T)) @ lambda_ ) + ydx.T @ lambda_)
        
        prob = cvxpy.Problem(objective,constraints)
        
        prob.solve(solver=cvxpy.OSQP, verbose=True, max_iter=40000)
        
        #-----------------------------------------------------------------------------

        lambda_v = lambda_.value
        
        w = w_count_and_record(lambda_v, y_1000_float64, X_1000_float64_1, inv_A, d)
        
        #-----------------------------------------------------------------------------
        
        np.save(Grid_name+'C_'+str(c)+'_Cr_'+str(c_r),w)
        
        print("Now count C = {0}, C_r = {1}\n".format(c,c_r))

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 120738882
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 40000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -3.3077e+03   1.14e+00   1.03e+01   1.00e-01   5.07e+02s
 200  -3.5874e+04   3.19e-02   8.85e-02   1.00e-01   5.75e+02s
 400  -3.5953e+04   5.75e-03   4.15

   1  -3.3077e+03   1.14e+00   1.03e+01   1.00e-01   5.07e+02s
 200  -2.0015e+05   3.05e-01   8.29e-01   1.00e-01   5.74e+02s
 400  -2.0479e+05   6.71e-02   4.00e-01   1.00e-01   6.41e+02s
 600  -2.0639e+05   5.57e-02   1.95e-01   1.00e-01   7.08e+02s
 800  -2.0721e+05   3.50e-02   1.36e-01   1.00e-01   7.74e+02s
1000  -2.0769e+05   1.88e-02   1.19e-01   1.00e-01   8.41e+02s
1200  -2.0799e+05   1.03e-02   1.03e-01   1.00e-01   9.07e+02s
1400  -2.0861e+05   1.22e-01   4.82e-02   1.82e-02   1.46e+03s
1600  -2.0875e+05   1.93e-02   3.17e-02   1.82e-02   1.52e+03s
1800  -2.0880e+05   1.54e-02   2.14e-02   1.82e-02   1.59e+03s
2000  -2.0883e+05   1.45e-02   1.93e-02   1.82e-02   1.65e+03s
2200  -2.0884e+05   1.83e-03   1.26e-02   1.82e-02   1.72e+03s
2400  -2.0885e+05   5.75e-03   1.24e-02   1.82e-02   1.79e+03s
2600  -2.0885e+05   2.48e-03   1.07e-02   1.82e-02   1.85e+03s
2800  -2.0886e+05   2.09e-04   7.89e-03   1.82e-02   1.92e+03s
3000  -2.0886e+05   5.09e-04   7.39e-03   1.82e-02   1.

2200  -4.7793e+05   7.20e-02   8.63e-02   1.49e-02   1.72e+03s
2400  -4.7825e+05   9.05e-02   4.25e-02   1.49e-02   1.79e+03s
2600  -4.7842e+05   7.26e-02   3.42e-02   1.49e-02   1.85e+03s
2800  -4.7852e+05   3.77e-02   3.29e-02   1.49e-02   1.92e+03s
3000  -4.7857e+05   4.82e-02   2.44e-02   1.49e-02   1.98e+03s
3200  -4.7861e+05   2.33e-02   2.01e-02   1.49e-02   2.05e+03s
3400  -4.7863e+05   1.92e-03   1.89e-02   1.49e-02   2.12e+03s
3600  -4.7865e+05   3.22e-02   1.43e-02   1.49e-02   2.18e+03s
3800  -4.7866e+05   1.63e-03   1.34e-02   1.49e-02   2.25e+03s
4000  -4.7867e+05   7.23e-04   1.27e-02   1.49e-02   2.32e+03s
4200  -4.7867e+05   3.28e-03   1.18e-02   1.49e-02   2.38e+03s
4400  -4.7868e+05   7.05e-05   1.12e-02   1.49e-02   2.45e+03s
4600  -4.7868e+05   5.69e-04   9.58e-03   1.49e-02   2.51e+03s
4800  -4.7868e+05   1.23e-04   8.10e-03   1.49e-02   2.58e+03s
5000  -4.7870e+05   8.65e-03   3.17e-03   8.94e-04   3.13e+03s
5200  -4.7870e+05   1.10e-03   1.84e-03   8.94e-04   3.

---

### Сетка 4: Сетка с всеми признаками от классификации и с flexibility
Признаки никак не обрабатывались. Ошибка в вычислениях d исправлена

$C_r$ in [0,5,15,30]

$C$ in [15,50,100,200,300,400,500,1000]

In [127]:
C_r = [0,5,15,30]
C = [15,50,100,200,300,400,500,1000]
Grid_name = "output_grids/Grid_4/Grid_4_"

In [128]:
ind = [i for i in range(6440)]
ind.append(6818)

X_1000_float64_1 = X_1000_float64[:,ind]

X_1000_float64_1.shape

(19000, 6441)

In [129]:
P = 1000
D = 19

set_i_and_j = range(1, P*D+1)
set_k = range(1, D+1)
set_l = range(0,P*D,D)

In [130]:
for c in C:
    for c_r in C_r:
        #----------------------------------------------------------------------------
        
        index_native_conf = index_with_native_conformation(y_1000_float64)
        
        X_1000_float64_native = X_1000_float64_1[index_native_conf]
        
        inv_A = inv_A_count_and_record( c_r, X_1000_float64_native )
        
        d = d_count_and_record( c_r, X_1000_float64_native, inv_A, s_1000_float64 )
        
        X_1000_float64_new = new_X_count_and_record( inv_A, X_1000_float64_1 )
        
        ydx = creat_and_record_vector_ydx( X_1000_float64_new, y_1000_float64, d )
        
        #-----------------------------------------------------------------------------

        lambda_ = cvxpy.Variable(P*D)

        constraints = [0 <= lambda_, lambda_ <= c]
        constraints += [ lambda_[i:i+D].T @ y_1000_float64[i:i+D] == 0 for i in set_l]
        
        objective = cvxpy.Minimize(0.5*cvxpy.sum_squares( (y_1000_float64 * (X_1000_float64_new.T)) @ lambda_ ) + ydx.T @ lambda_)
        
        prob = cvxpy.Problem(objective,constraints)
        
        prob.solve(solver=cvxpy.OSQP, verbose=True, max_iter=40000)
        
        #-----------------------------------------------------------------------------

        lambda_v = lambda_.value
        
        w = w_count_and_record(lambda_v, y_1000_float64, X_1000_float64_1, inv_A, d)
        
        #-----------------------------------------------------------------------------
        
        np.save(Grid_name+'C_'+str(c)+'_Cr_'+str(c_r),w)
        
        print("Now count C = {0}, C_r = {1}\n".format(c,c_r))

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 13034540
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 40000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -1.0722e+04   3.71e-01   2.44e+00   1.00e-01   3.51e+01s
 200  -2.9834e+04   1.46e-02   5.35e-02   1.00e-01   4.97e+01s
 400  -2.9852e+04   1.03e-02   4.56e

   1  -1.0722e+04   3.71e-01   2.44e+00   1.00e-01   3.51e+01s
 200  -9.8018e+04   8.36e-02   1.70e+00   1.00e-01   4.89e+01s
 400  -9.8530e+04   7.09e-02   8.46e-02   1.40e-02   9.52e+01s
 600  -9.8553e+04   8.08e-03   4.81e-02   1.40e-02   1.09e+02s
 800  -9.8560e+04   3.31e-02   2.99e-02   1.40e-02   1.22e+02s
1000  -9.8562e+04   2.19e-02   1.54e-02   1.40e-02   1.36e+02s
1200  -9.8563e+04   3.83e-03   1.40e-02   1.40e-02   1.49e+02s
1400  -9.8564e+04   1.62e-03   1.30e-02   1.40e-02   1.63e+02s
1600  -9.8565e+04   6.99e-04   1.24e-02   1.40e-02   1.76e+02s
1800  -9.8566e+04   1.49e-02   4.58e-03   2.61e-03   2.22e+02s
2000  -9.8566e+04   4.59e-03   3.06e-03   2.61e-03   2.35e+02s
2200  -9.8566e+04   2.87e-04   2.40e-03   2.61e-03   2.49e+02s
2400  -9.8566e+04   2.26e-03   1.10e-03   4.86e-04   2.94e+02s
2600  -9.8566e+04   5.01e-04   3.71e-04   4.86e-04   3.08e+02s
2800  -9.8566e+04   2.86e-04   1.89e-04   4.86e-04   3.22e+02s
2950  -9.8566e+04   1.71e-04   1.10e-04   4.86e-04   3.

3400  -1.1602e+05   1.44e-05   3.65e-03   1.72e-02   2.13e+03s
3600  -1.1602e+05   2.59e-04   3.31e-03   1.72e-02   2.19e+03s
3800  -1.1602e+05   8.64e-03   1.75e-03   6.98e-04   2.74e+03s
3950  -1.1602e+05   4.28e-04   1.77e-04   6.98e-04   2.79e+03s

status:               solved
solution polish:      unsuccessful
number of iterations: 3950
optimal objective:    -116022.4689
run time:             3.32e+03s
optimal rho estimate: 7.85e-04

Now count C = 50, C_r = 30

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 13034540
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0

Now count C = 100, C_r = 15

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 120738882
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 40000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -4.0695e+03   1.49e+00   9.79e+00   1.00e-01   5.06e+02s
 200  -2.0875e+05   2.56e-01   7.52e-01   1.00e-01   5.73e+02s
 400  

 600  -3.8441e+05   1.21e-01   4.99e-01   1.00e-01   7.06e+02s
 800  -3.8789e+05   5.06e-02   4.27e-01   1.00e-01   7.72e+02s
1000  -3.8981e+05   5.36e-02   2.21e-01   1.00e-01   8.38e+02s
1200  -3.9108e+05   3.04e-02   2.24e-01   1.00e-01   9.05e+02s
1400  -3.9393e+05   1.54e-01   9.35e-02   1.90e-02   1.45e+03s
1600  -3.9469e+05   6.79e-02   6.35e-02   1.90e-02   1.52e+03s
1800  -3.9502e+05   6.64e-02   4.86e-02   1.90e-02   1.59e+03s
2000  -3.9519e+05   3.49e-02   3.70e-02   1.90e-02   1.65e+03s
2200  -3.9528e+05   4.53e-02   3.17e-02   1.90e-02   1.72e+03s
2400  -3.9534e+05   3.59e-02   2.79e-02   1.90e-02   1.78e+03s
2600  -3.9537e+05   1.24e-02   2.26e-02   1.90e-02   1.85e+03s
2800  -3.9539e+05   2.13e-03   2.18e-02   1.90e-02   1.92e+03s
3000  -3.9541e+05   1.92e-03   1.69e-02   1.90e-02   1.98e+03s
3200  -3.9545e+05   8.04e-02   1.02e-02   3.26e-03   2.53e+03s
3400  -3.9546e+05   3.08e-03   6.81e-03   3.26e-03   2.60e+03s
3600  -3.9546e+05   2.34e-03   5.28e-03   3.26e-03   2.

3800  -5.5756e+05   1.74e-02   2.58e-02   1.82e-02   2.24e+03s
4000  -5.5759e+05   6.53e-03   2.33e-02   1.82e-02   2.31e+03s
4200  -5.5761e+05   4.18e-03   1.77e-02   1.82e-02   2.37e+03s
4400  -5.5764e+05   6.14e-02   1.73e-02   2.96e-03   2.92e+03s
4600  -5.5768e+05   9.41e-03   1.01e-02   2.96e-03   2.99e+03s
4800  -5.5769e+05   5.18e-03   6.32e-03   2.96e-03   3.06e+03s
5000  -5.5770e+05   4.49e-02   5.48e-03   2.96e-03   3.12e+03s
5200  -5.5770e+05   1.81e-03   4.71e-03   2.96e-03   3.19e+03s
5400  -5.5770e+05   8.09e-03   4.07e-03   2.96e-03   3.25e+03s
5600  -5.5771e+05   4.82e-04   3.50e-03   2.96e-03   3.32e+03s
5800  -5.5771e+05   1.58e-02   7.49e-04   4.65e-04   3.87e+03s
5850  -5.5771e+05   2.07e-03   6.41e-04   4.65e-04   3.88e+03s

status:               solved
solution polish:      unsuccessful
number of iterations: 5850
optimal objective:    -557706.8102
run time:             4.41e+03s
optimal rho estimate: 3.89e-04

Now count C = 300, C_r = 5

-------------------------

3800  -7.0953e+05   9.45e-03   3.83e-03   3.52e-04   3.86e+02s
4000  -7.0954e+05   3.49e-03   1.32e-03   3.52e-04   4.00e+02s
4125  -7.0954e+05   1.73e-03   7.31e-04   3.52e-04   4.08e+02s

status:               solved
solution polish:      unsuccessful
number of iterations: 4125
optimal objective:    -709537.0119
run time:             4.41e+02s
optimal rho estimate: 2.83e-04

Now count C = 400, C_r = 0

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 120738882
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1

3800  -7.3839e+05   3.50e-03   4.69e-03   2.61e-03   2.72e+03s
4000  -7.3839e+05   3.50e-03   3.96e-03   2.61e-03   2.78e+03s
4200  -7.3839e+05   3.80e-03   2.99e-03   2.61e-03   2.85e+03s
4400  -7.3839e+05   2.11e-04   2.91e-03   2.61e-03   2.92e+03s
4600  -7.3839e+05   1.51e-04   2.84e-03   2.61e-03   2.98e+03s
4800  -7.3840e+05   1.47e-03   2.77e-03   2.61e-03   3.05e+03s
5000  -7.3840e+05   2.79e-04   2.70e-03   2.61e-03   3.11e+03s
5200  -7.3840e+05   7.38e-05   2.61e-03   2.61e-03   3.18e+03s
5400  -7.3840e+05   1.15e-02   2.53e-03   2.61e-03   3.24e+03s
5600  -7.3840e+05   3.61e-04   2.45e-03   2.61e-03   3.31e+03s
5800  -7.3840e+05   4.04e-05   2.37e-03   2.61e-03   3.37e+03s
6000  -7.3840e+05   3.21e-05   2.30e-03   2.61e-03   3.44e+03s
6125  -7.3840e+05   2.28e-03   1.87e-04   1.39e-04   3.96e+03s

status:               solved
solution polish:      unsuccessful
number of iterations: 6125
optimal objective:    -738396.4239
run time:             4.49e+03s
optimal rho estimate: 

4200  -8.7937e+05   1.98e-03   1.54e-02   1.33e-02   2.37e+03s
4400  -8.7944e+05   2.78e-02   1.30e-02   2.04e-03   2.92e+03s
4600  -8.7946e+05   5.12e-03   7.29e-03   2.04e-03   2.99e+03s
4800  -8.7946e+05   1.27e-03   5.69e-03   2.04e-03   3.05e+03s
5000  -8.7947e+05   5.09e-04   3.73e-03   2.04e-03   3.12e+03s
5200  -8.7947e+05   4.61e-04   3.08e-03   2.04e-03   3.19e+03s
5400  -8.7947e+05   2.03e-04   2.14e-03   2.04e-03   3.25e+03s
5600  -8.7947e+05   1.50e-03   8.29e-04   2.68e-04   3.80e+03s

status:               solved
solution polish:      unsuccessful
number of iterations: 5600
optimal objective:    -879468.9269
run time:             4.34e+03s
optimal rho estimate: 1.54e-04

Now count C = 500, C_r = 15

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-------------------------------------------

4400  -1.5317e+06   1.25e-03   4.28e-03   1.91e-03   2.92e+03s
4600  -1.5317e+06   2.37e-03   4.03e-03   1.91e-03   2.99e+03s
4800  -1.5317e+06   6.09e-03   3.65e-03   1.91e-03   3.05e+03s
5000  -1.5317e+06   4.64e-04   3.30e-03   1.91e-03   3.12e+03s
5200  -1.5317e+06   1.87e-04   2.91e-03   1.91e-03   3.18e+03s
5400  -1.5317e+06   2.14e-04   2.54e-03   1.91e-03   3.25e+03s
5500  -1.5317e+06   5.27e-03   1.18e-03   2.09e-04   3.76e+03s

status:               solved
solution polish:      unsuccessful
number of iterations: 5500
optimal objective:    -1531674.3896
run time:             4.29e+03s
optimal rho estimate: 1.66e-04

Now count C = 1000, C_r = 5

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constrai

---

### Сетка 5: Сетка с всеми признаками от классификации и с flexibility
Признаки никак не обрабатывались. Ошибка в вычислениях d исправлена

$C_r$ in [50,100,150,250,500,700,1000,1200,1500,1700,2000,2500]

$C$ in [10000]

In [142]:
C_r = [50,100,150,250,500,700,1000,1200,1500,1700,2000,2500,3000,4000,5000,6000,7000,8000]
C = [10000]
Grid_name = "output_grids/Grid_5/Grid_5_"

In [143]:
ind = [i for i in range(6440)]
ind.append(6818)

X_1000_float64_1 = X_1000_float64[:,ind]

X_1000_float64_1.shape

(19000, 6441)

In [144]:
P = 1000
D = 19

set_i_and_j = range(1, P*D+1)
set_k = range(1, D+1)
set_l = range(0,P*D,D)

In [145]:
for c in C:
    for c_r in C_r:
        #----------------------------------------------------------------------------
        
        index_native_conf = index_with_native_conformation(y_1000_float64)
        
        X_1000_float64_native = X_1000_float64_1[index_native_conf]
        
        inv_A = inv_A_count_and_record( c_r, X_1000_float64_native )
        
        d = d_count_and_record( c_r, X_1000_float64_native, inv_A, s_1000_float64 )
        
        X_1000_float64_new = new_X_count_and_record( inv_A, X_1000_float64_1 )
        
        ydx = creat_and_record_vector_ydx( X_1000_float64_new, y_1000_float64, d )
        
        #-----------------------------------------------------------------------------

        lambda_ = cvxpy.Variable(P*D)

        constraints = [0 <= lambda_, lambda_ <= c]
        constraints += [ lambda_[i:i+D].T @ y_1000_float64[i:i+D] == 0 for i in set_l]
        
        objective = cvxpy.Minimize(0.5*cvxpy.sum_squares( (y_1000_float64 * (X_1000_float64_new.T)) @ lambda_ ) + ydx.T @ lambda_)
        
        prob = cvxpy.Problem(objective,constraints)
        
        prob.solve(solver=cvxpy.OSQP, verbose=True, max_iter=40000)
        
        #-----------------------------------------------------------------------------

        lambda_v = lambda_.value
        
        w = w_count_and_record(lambda_v, y_1000_float64, X_1000_float64_1, inv_A, d)
        
        #-----------------------------------------------------------------------------
        
        np.save(Grid_name+'C_'+str(c)+'_Cr_'+str(c_r),w)
        
        print("Now count C = {0}, C_r = {1}\n".format(c,c_r))

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 120738882
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 40000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -1.3720e+04   2.59e+00   1.07e+01   1.00e-01   5.07e+02s
 200  -7.2911e+05   1.24e-01   4.37e+00   1.00e-01   5.74e+02s
 400  -1.2152e+06   1.47e-01   3.72

4800  -1.3012e+07   1.71e+00   9.09e-02   8.00e-04   3.05e+03s
5000  -1.3017e+07   9.93e-01   6.62e-02   8.00e-04   3.12e+03s
5200  -1.3020e+07   4.68e-01   5.66e-02   8.00e-04   3.18e+03s
5400  -1.3022e+07   4.91e-02   4.71e-02   8.00e-04   3.25e+03s
5600  -1.3023e+07   1.64e-01   4.42e-02   8.00e-04   3.32e+03s
5800  -1.3024e+07   2.01e-02   3.97e-02   8.00e-04   3.38e+03s
6000  -1.3024e+07   9.00e-03   2.48e-02   8.00e-04   3.45e+03s
6200  -1.3025e+07   1.11e-01   1.25e-02   1.07e-04   3.99e+03s
6400  -1.3025e+07   5.50e-01   8.29e-03   1.07e-04   4.06e+03s
6600  -1.3026e+07   1.42e-02   7.28e-03   1.07e-04   4.13e+03s
6800  -1.3026e+07   6.34e-03   5.02e-03   1.07e-04   4.19e+03s
6825  -1.3026e+07   5.86e-03   4.89e-03   1.07e-04   4.20e+03s

status:               solved
solution polish:      unsuccessful
number of iterations: 6825
optimal objective:    -13025536.3080
run time:             4.74e+03s
optimal rho estimate: 2.62e-05

Now count C = 10000, C_r = 5000

------------------

 200  -9.4389e+05   2.30e-01   5.54e+00   1.00e-01   5.72e+02s
 400  -1.6043e+06   2.17e-01   4.95e+00   1.00e-01   6.38e+02s
 600  -2.1346e+06   7.36e-02   4.77e+00   1.00e-01   7.03e+02s
 800  -4.5370e+06   2.99e+00   3.60e+00   1.30e-02   1.25e+03s
1000  -6.3015e+06   1.16e+00   2.92e+00   1.30e-02   1.32e+03s
1200  -7.6720e+06   3.85e-01   2.71e+00   1.30e-02   1.38e+03s
1400  -1.3311e+07   3.32e+01   1.11e+00   1.13e-03   1.93e+03s
1600  -1.4281e+07   7.08e+00   4.74e-01   1.13e-03   1.99e+03s
1800  -1.4465e+07   2.99e+00   3.21e-01   1.13e-03   2.06e+03s
2000  -1.4530e+07   1.88e+00   2.15e-01   1.13e-03   2.13e+03s
2200  -1.4561e+07   2.69e+00   1.62e-01   1.13e-03   2.19e+03s
2400  -1.4577e+07   3.29e+00   1.42e-01   1.13e-03   2.26e+03s
2600  -1.4585e+07   2.29e+00   9.12e-02   1.13e-03   2.32e+03s
2800  -1.4590e+07   1.37e+00   7.38e-02   1.13e-03   2.39e+03s
3000  -1.4593e+07   1.26e-01   6.49e-02   1.13e-03   2.46e+03s
3200  -1.4595e+07   1.88e-01   5.61e-02   1.13e-03   2.

---

### Сетка 6.1: Сетка с всеми признаками от классификации и с flexibility
Признаки никак не обрабатывались. Ошибка в вычислениях d исправлена

$C_r$ in $10\left(\sqrt{10}\right)^{[1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 5.6, 5.8]}$

$C$ in $100\left(\sqrt{10}\right)^{[1,2,3,4,5,6]}$

In [173]:
C_r = np.round(10*np.sqrt(10)**np.array([1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,5.6,5.8]),0)
C = np.round(100*np.sqrt(10)**np.array([1,2,3,4,5,6]),0)
Grid_name = "output_grids/Grid_6_1/Grid_6_1_"

In [174]:
print(C_r)
print(C)

[  32.   56.  100.  178.  316.  562. 1000. 1778. 3162. 5623. 6310. 7943.]
[   316.   1000.   3162.  10000.  31623. 100000.]


In [175]:
ind = [i for i in range(6440)]
ind.append(6818)

X_1000_float64_1 = X_1000_float64[:,ind]

X_1000_float64_1.shape

(19000, 6441)

In [176]:
P = 1000
D = 19

set_i_and_j = range(1, P*D+1)
set_k = range(1, D+1)
set_l = range(0,P*D,D)

In [177]:
index_native_conf = index_with_native_conformation(y_1000_float64)

X_1000_float64_native = X_1000_float64_1[index_native_conf]

for c_r in C_r:
    
    #----------------------------------------------------------------------------
    
    inv_A = inv_A_count_and_record( c_r, X_1000_float64_native )

    d = d_count_and_record( c_r, X_1000_float64_native, inv_A, s_1000_float64 )

    X_1000_float64_new = new_X_count_and_record( inv_A, X_1000_float64_1 )

    ydx = creat_and_record_vector_ydx( X_1000_float64_new, y_1000_float64, d )
    
    #-----------------------------------------------------------------------------
    
    for c in C:
        
        if c_r >= c:
            continue
        
        #-----------------------------------------------------------------------------
        
        lambda_ = cvxpy.Variable(P*D)

        constraints = [0 <= lambda_, lambda_ <= c]
        constraints += [ lambda_[i:i+D].T @ y_1000_float64[i:i+D] == 0 for i in set_l]
        
        objective = cvxpy.Minimize(0.5*cvxpy.sum_squares( (y_1000_float64 * (X_1000_float64_new.T)) @ lambda_ ) + ydx.T @ lambda_)
        
        prob = cvxpy.Problem(objective,constraints)
        
        prob.solve(solver=cvxpy.OSQP, verbose=True, max_iter=40000)
        
        #-----------------------------------------------------------------------------

        lambda_v = lambda_.value
        
        w = w_count_and_record(lambda_v, y_1000_float64, X_1000_float64_1, inv_A, d)
        
        #-----------------------------------------------------------------------------
        
        np.save(Grid_name+'C_'+str(c)+'_Cr_'+str(c_r),w)
        
        print("Now count C = {0}, C_r = {1}\n".format(c,c_r))

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 120757882
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 40000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -4.1088e+03   1.51e+00   9.77e+00   1.00e-01   5.07e+02s
 200  -3.3563e+05   1.10e-01   2.09e+00   1.00e-01   5.73e+02s
 400  -5.3433e+05   6.35e-01   1.30

5200  -3.6371e+06   1.60e-01   1.05e-02   5.02e-04   3.18e+03s
5400  -3.6371e+06   2.90e-02   5.60e-03   5.02e-04   3.25e+03s
5600  -3.6371e+06   3.42e-03   4.04e-03   5.02e-04   3.31e+03s
5800  -3.6371e+06   1.21e-02   2.48e-03   5.02e-04   3.38e+03s
5825  -3.6371e+06   1.01e-02   2.38e-03   5.02e-04   3.39e+03s

status:               solved
solution polish:      unsuccessful
number of iterations: 5825
optimal objective:    -3637126.3933
run time:             3.92e+03s
optimal rho estimate: 2.88e-04

Now count C = 3162.0, C_r = 32.0

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 120757882
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, 

Now count C = 100000.0, C_r = 32.0

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 120662882
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 40000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -4.5126e+03   1.62e+00   9.54e+00   1.00e-01   5.05e+02s
 200  -3.4629e+05   1.69e-01   2.14e+00   1.00e-01   5.73e+02s

 200  -3.4629e+05   1.69e-01   2.14e+00   1.00e-01   5.72e+02s
 400  -5.8879e+05   7.58e-02   2.13e+00   1.00e-01   6.38e+02s
 600  -7.9743e+05   1.20e-01   2.17e+00   1.00e-01   7.05e+02s
 800  -9.8573e+05   5.17e-02   2.19e+00   1.00e-01   7.71e+02s
1000  -1.1596e+06   1.35e-01   2.30e+00   1.00e-01   8.38e+02s
1200  -1.3225e+06   3.88e-02   2.31e+00   1.00e-01   9.04e+02s
1400  -4.1721e+06   3.94e+00   2.43e+00   3.87e-03   1.45e+03s
1600  -5.9993e+06   1.06e+01   1.53e+00   3.87e-03   1.52e+03s
1800  -6.8809e+06   1.11e+01   9.28e-01   3.87e-03   1.59e+03s
2000  -7.2587e+06   3.33e+00   6.79e-01   3.87e-03   1.65e+03s
2200  -7.4438e+06   6.00e+00   4.82e-01   3.87e-03   1.72e+03s
2400  -7.5487e+06   3.53e+00   3.92e-01   3.87e-03   1.78e+03s
2600  -7.6149e+06   3.97e+00   3.30e-01   3.87e-03   1.85e+03s
2800  -7.6569e+06   4.22e-01   2.78e-01   3.87e-03   1.92e+03s
3000  -7.6838e+06   1.17e+00   2.38e-01   3.87e-03   1.98e+03s
3200  -7.7030e+06   2.28e-01   2.27e-01   3.87e-03   2.

1600  -6.3301e+05   6.09e-02   2.82e-01   1.00e-01   1.04e+03s
1800  -6.3434e+05   6.25e-02   2.58e-01   1.00e-01   1.11e+03s
2000  -6.3536e+05   2.24e-02   2.29e-01   1.00e-01   1.18e+03s
2200  -6.3613e+05   1.98e-02   2.01e-01   1.00e-01   1.24e+03s
2400  -6.3674e+05   1.05e-02   1.76e-01   1.00e-01   1.31e+03s
2600  -6.3914e+05   2.02e-01   9.27e-02   1.21e-02   1.86e+03s
2800  -6.3985e+05   1.24e-01   5.78e-02   1.21e-02   1.92e+03s
3000  -6.4016e+05   1.63e-01   5.10e-02   1.21e-02   1.99e+03s
3200  -6.4032e+05   1.25e-02   3.56e-02   1.21e-02   2.06e+03s
3400  -6.4042e+05   4.12e-02   2.71e-02   1.21e-02   2.12e+03s
3600  -6.4048e+05   1.42e-02   2.41e-02   1.21e-02   2.19e+03s
3800  -6.4052e+05   8.24e-04   2.00e-02   1.21e-02   2.26e+03s
4000  -6.4054e+05   3.49e-03   1.61e-02   1.21e-02   2.32e+03s
4200  -6.4056e+05   2.03e-02   1.30e-02   1.21e-02   2.39e+03s
4400  -6.4057e+05   1.42e-02   1.25e-02   1.21e-02   2.46e+03s
4600  -6.4058e+05   2.07e-03   1.14e-02   1.21e-02   2.

...

1200  -6.8269e+06   1.33e+00   2.53e+00   7.41e-03   1.39e+03s
1400  -1.3341e+07   1.83e+01   1.93e+00   9.42e-04   1.94e+03s
1600  -1.6775e+07   3.98e+01   1.11e+00   9.42e-04   2.01e+03s
1800  -1.8125e+07   3.29e+01   6.89e-01   9.42e-04   2.08e+03s
2000  -1.8693e+07   1.22e+01   4.88e-01   9.42e-04   2.14e+03s
2200  -1.8960e+07   1.96e+00   3.99e-01   9.42e-04   2.21e+03s
2400  -1.9108e+07   3.02e+00   2.87e-01   9.42e-04   2.27e+03s
2600  -1.9194e+07   3.01e+00   2.51e-01   9.42e-04   2.34e+03s
2800  -1.9249e+07   1.35e+00   1.94e-01   9.42e-04   2.41e+03s
3000  -1.9287e+07   1.47e+00   1.78e-01   9.42e-04   2.47e+03s
3200  -1.9313e+07   1.99e+00   1.65e-01   9.42e-04   2.54e+03s
3400  -1.9331e+07   7.20e-01   1.22e-01   9.42e-04   2.61e+03s
3600  -1.9344e+07   8.95e-01   1.12e-01   9.42e-04   2.67e+03s
3800  -1.9354e+07   1.05e+00   1.03e-01   9.42e-04   2.74e+03s
4000  -1.9361e+07   1.82e-01   8.78e-02   9.42e-04   2.80e+03s
4200  -1.9367e+07   2.15e-01   8.37e-02   9.42e-04   2.

   1  -1.3977e+04   2.61e+00   1.08e+01   1.00e-01   5.09e+02s
 200  -7.3967e+05   7.75e-02   4.44e+00   1.00e-01   5.76e+02s
 400  -1.2339e+06   9.39e-02   3.78e+00   1.00e-01   6.44e+02s
 600  -1.6282e+06   1.04e-01   3.61e+00   1.00e-01   7.12e+02s
 800  -5.0536e+06   1.97e+00   2.38e+00   6.07e-03   1.26e+03s
1000  -7.0650e+06   1.81e+00   2.42e+00   6.07e-03   1.33e+03s
1200  -8.6362e+06   2.92e+00   2.48e+00   6.07e-03   1.40e+03s
1400  -1.4364e+07   1.13e+01   1.88e+00   1.14e-03   1.95e+03s
1600  -1.7817e+07   1.38e+01   1.30e+00   1.14e-03   2.01e+03s
1800  -1.9523e+07   8.67e+00   9.55e-01   1.14e-03   2.08e+03s
2000  -2.0317e+07   4.89e+00   6.37e-01   1.14e-03   2.14e+03s
2200  -2.0732e+07   4.95e+00   5.46e-01   1.14e-03   2.21e+03s
2400  -2.0966e+07   8.05e+00   3.88e-01   1.14e-03   2.28e+03s
2600  -2.1107e+07   4.21e+00   2.92e-01   1.14e-03   2.34e+03s
2800  -2.1199e+07   1.89e+00   2.64e-01   1.14e-03   2.41e+03s
3000  -2.1263e+07   6.67e-01   2.50e-01   1.14e-03   2.

Now count C = 10000.0, C_r = 5623.0

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 120738882
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 40000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -1.7047e+04   2.86e+00   1.11e+01   1.00e-01   5.05e+02s
 200  -8.6608e+05   1.23e-01   5.12e+00   1.00e-01   5.72e+02

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25441, constraints m = 45441
          nnz(P) + nnz(A) = 120738882
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 40000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -1.7684e+04   2.92e+00   1.12e+01   1.00e-01   5.09e+02s
 200  -8.9227e+05   1.09e-01   5.26e+00   1.00e-01   5.77e+02s
 400  -1.5081e+06   1.06e-01   4.68

4800  -1.4495e+07   6.98e-01   2.18e-01   1.67e-03   3.06e+03s
5000  -1.4522e+07   5.71e-01   1.80e-01   1.67e-03   3.13e+03s
5200  -1.4538e+07   1.64e+00   1.60e-01   1.67e-03   3.19e+03s
5400  -1.4549e+07   4.66e-01   1.45e-01   1.67e-03   3.26e+03s
5600  -1.4556e+07   3.97e-01   1.17e-01   1.67e-03   3.32e+03s
5800  -1.4560e+07   3.89e-01   8.99e-02   1.67e-03   3.39e+03s
6000  -1.4563e+07   2.59e-01   7.58e-02   1.67e-03   3.46e+03s
6200  -1.4566e+07   1.23e-01   6.96e-02   1.67e-03   3.52e+03s
6400  -1.4567e+07   3.30e-02   6.36e-02   1.67e-03   3.59e+03s
6600  -1.4568e+07   3.26e-02   5.72e-02   1.67e-03   3.66e+03s
6800  -1.4569e+07   1.75e-02   5.05e-02   1.67e-03   3.72e+03s
7000  -1.4572e+07   2.17e+00   3.43e-02   2.26e-04   4.27e+03s
7200  -1.4573e+07   4.87e-01   1.92e-02   2.26e-04   4.34e+03s
7400  -1.4573e+07   1.89e-01   1.53e-02   2.26e-04   4.40e+03s
7600  -1.4573e+07   1.35e-01   1.17e-02   2.26e-04   4.47e+03s
7800  -1.4574e+07   6.49e-03   8.74e-03   2.26e-04   4.

In [180]:
for n in range(1,8):
    print(np.sqrt(10)**n)

3.1622776601683795
10.000000000000002
31.6227766016838
100.00000000000003
316.227766016838
1000.0000000000003
3162.277660168381


---

### Сетка 7: Сетка с всеми признаками от классификации и регрессии
Признаки никак не обрабатывались. Ошибка в вычислениях d исправлена

$C_r$ in $\left(\sqrt{10}\right)^{[1,2,3,4,4.5,5,6,7]}$

$C$ in $\left(\sqrt{10}\right)^{[1,2,3,4,4.5,5,6,7]}$

In [9]:
C_r = np.round(np.sqrt(10)**np.array([1,2,3,4,4.5,5,6,7]),3)
C = np.round(np.sqrt(10)**np.array([1,2,3,4,4.5,5,6,7]),3)
Grid_name = "output_grids/Grid_7/Grid_7_"

In [10]:
print(C_r)
print(C)

[3.162000e+00 1.000000e+01 3.162300e+01 1.000000e+02 1.778280e+02
 3.162280e+02 1.000000e+03 3.162278e+03]
[3.162000e+00 1.000000e+01 3.162300e+01 1.000000e+02 1.778280e+02
 3.162280e+02 1.000000e+03 3.162278e+03]


In [331]:
path = '/disk2/chmnk/pl-opt/data/pdbbind/2016/protlig_stat/for_optimization/general_t14_t3_l7.0_g1.0_r1.0sasa_solvent_all_float64_seed104504/general_t14_t3_l7.0_g1.0_r1.0sasa_solvent_all_float64_seed104504_from0_to1000'


y_1000_float64 = np.load(path+"/labels_general_t14_t3_l7.0_g1.0_r1.0sasa_solvent_all_float64_seed104504_from0_to1000.npy").astype(dtype=np.float)
s_1000_float64 = np.load(path+"/affinities_general_t14_t3_l7.0_g1.0_r1.0sasa_solvent_all_float64_seed104504_from0_to1000.npy")

loader = np.load(path+"/data_array_general_t14_t3_l7.0_g1.0_r1.0sasa_solvent_all_float64_seed104504_from0_to1000_sparse.npz")
X_1000_float64_1 = csr_matrix((loader['data'], loader['indices'], loader['indptr']),
               shape=loader['shape'])[:,:3400]
del loader

X_1000_float64_1.shape

(19000, 3400)

In [333]:
P = 1000
D = 19

set_i_and_j = range(1, P*D+1)
set_k = range(1, D+1)
set_l = range(0,P*D,D)

In [327]:
index_native_conf = index_with_native_conformation(y_1000_float64)

X_1000_float64_native = X_1000_float64_1[index_native_conf]

C_r = 1000*C_r/(X_1000_float64_native.shape[0])

for c_r in C_r:
    
    #----------------------------------------------------------------------------
    
    inv_A = inv_A_count_and_record( c_r, X_1000_float64_native )

    d = d_count_and_record( c_r, X_1000_float64_native, inv_A, s_1000_float64 )

    X_1000_float64_new = new_X_count_and_record( inv_A, X_1000_float64_1 )

    ydx = creat_and_record_vector_ydx( X_1000_float64_new, y_1000_float64, d )
    
    #-----------------------------------------------------------------------------
    
    for c in C:
        
        if c_r >= c:
            continue
        
        #-----------------------------------------------------------------------------
        
        lambda_ = cvxpy.Variable(P*D)

        constraints = [0 <= lambda_, lambda_ <= c]
        constraints += [ lambda_[i:i+D].T @ y_1000_float64[i:i+D] == 0 for i in set_l]
        
        objective = cvxpy.Minimize(0.5*cvxpy.sum_squares( (y_1000_float64 * (X_1000_float64_new.T)) @ lambda_ ) + ydx.T @ lambda_)
        
        prob = cvxpy.Problem(objective,constraints)
        
        prob.solve(solver=cvxpy.OSQP, verbose=True, max_iter=40000)
        
        #-----------------------------------------------------------------------------

        lambda_v = lambda_.value
        
        w = w_count_and_record(lambda_v, y_1000_float64, X_1000_float64_1, inv_A, d)
        
        #-----------------------------------------------------------------------------
        
        np.save(Grid_name+'C_'+str(c)+'_Cr_'+str(c_r),w)
        
        print("Now count C = {0}, C_r = {1}\n".format(c,c_r))

KeyboardInterrupt: 

---

## Проверка оптимизации

In [8]:
import psutil, resource

In [21]:
def bring_blocky_diagonal_XXT_np(X, n_conformation = 19):
    
    row_csc =  []
    col_csc =  []
    data_csc = []
    
    shape1 = X.shape[0]
    shape2 = X.shape[1]
    
    n_complex = -1
    k_conformation = -1
    
    for ind_row, row in enumerate(X):
        k_conformation += 1
        
        if (k_conformation) % n_conformation == 0:
            n_complex+=1
            
        for ind_col in range( (n_complex)*(n_conformation), (n_complex+1)*(n_conformation) ):
            row_csc.append(ind_row)
            col_csc.append(ind_col)
            data_csc.append(row @ X[ind_col])
            
    return csc_matrix((data_csc, (row_csc, col_csc)), shape=(shape1, shape1))

In [18]:
C = 100
C_r = 10

P = 1000
D = 19

set_i_and_j = range(1, P*D+1)
set_k = range(1, D+1)
set_l = range(0,P*D,D)

In [11]:
y_1000_float64 = np.load("Data/1000_64float_good/labels_general_t14_t3_l7.0_g1.0_r1.0sasa_solvent_all_float64_first1000.npy").astype(dtype=np.float)
s_1000_float64 = np.load("Data/1000_64float_good/affinities_general_t14_t3_l7.0_g1.0_r1.0sasa_solvent_all_float64_first1000.npy")

loader = np.load("Data/1000_64float_good/data_array_general_t14_t3_l7.0_g1.0_r1.0sasa_solvent_all_float64_first1000_sparse.npz")
X_1000_float64 = csr_matrix((loader['data'], loader['indices'], loader['indptr']),
               shape=loader['shape']) 
del loader

X_1000_float64_1 = X_1000_float64

X_1000_float64_1.shape

(19000, 6819)

In [13]:
print('The maximum memory consumption in the process = {0:0.3f} (Gb)\n'.format(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024 / 1024))

The maximum memory consumption in the process = 0.325 (Gb)



In [19]:
index_native_conf = index_with_native_conformation(y_1000_float64)

X_1000_float64_native = X_1000_float64_1[index_native_conf]

inv_A = inv_A_count_and_record( C_r, X_1000_float64_native )

d = d_count_and_record( C_r, X_1000_float64_native, inv_A, s_1000_float64 )

X_1000_float64_new = new_X_count_and_record( inv_A, X_1000_float64_1 )

ydx = creat_and_record_vector_ydx( X_1000_float64_new, y_1000_float64, d )

In [20]:
print('The maximum memory consumption in the process = {0:0.3f} (Gb)\n'.format(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024 / 1024))

The maximum memory consumption in the process = 2.936 (Gb)



In [22]:
XXT = bring_blocky_diagonal_XXT_np((y_1000_float64*X_1000_float64_new.T).T)

In [23]:
print('The maximum memory consumption in the process = {0:0.3f} (Gb)\n'.format(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024 / 1024))

The maximum memory consumption in the process = 3.041 (Gb)



In [25]:
lambda_ = cvxpy.Variable(P*D)

constraints = [0 <= lambda_, lambda_ <= C]
constraints += [ lambda_[i:i+D].T @ y_1000_float64[i:i+D] == 0 for i in set_l]

objective = cvxpy.Minimize(0.5*cvxpy.quad_form( lambda_, XXT ) + ydx.T @ lambda_)

prob = cvxpy.Problem(objective,constraints)

prob.solve(solver=cvxpy.OSQP, verbose=True, max_iter=40000)

print('The maximum memory consumption in the process = {0:0.3f} (Gb)\n'.format(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024 / 1024))

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 19000, constraints m = 39000
          nnz(P) + nnz(A) = 247000
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 40000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -3.2519e+03   8.51e-01   9.16e+00   1.00e-01   3.92e-02s
 200  -2.3186e+05   9.94e-02   2.30e-02   6.42e-03   2.73e-01s
 400  -2.3189e+05   2.21e-03   9.88e-0

In [26]:
lambda_v = lambda_.value

#Забыл, что надо записывать w, поэтому на первой итерации у меня записаны lambda
w = w_count_and_record(lambda_v, y_1000_float64, X_1000_float64_1, inv_A, d)

np.save('block_matrix',w)

In [27]:
w

array([-0.06643846, -0.84368781, -4.46767769, ...,  4.32134552,
        3.2080595 , -0.01876835])

---

In [28]:
print('The maximum memory consumption in the process = {0:0.3f} (Gb)\n'.format(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024 / 1024))

The maximum memory consumption in the process = 7.552 (Gb)



In [ ]:
lambda_ = cvxpy.Variable(P*D)

constraints = [0 <= lambda_, lambda_ <= C]
constraints += [ lambda_[i:i+D].T @ y_1000_float64[i:i+D] == 0 for i in set_l]

objective = cvxpy.Minimize(0.5*cvxpy.sum_squares( (y_1000_float64 * (X_1000_float64_new.T)) @ lambda_) + ydx.T @ lambda_)

prob = cvxpy.Problem(objective,constraints)

prob.solve(solver=cvxpy.OSQP, verbose=True, max_iter=40000)

print('The maximum memory consumption in the process = {0:0.3f} (Gb)\n'.format(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024 / 1024))

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 25819, constraints m = 45819
          nnz(P) + nnz(A) = 129631638
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 40000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -3.1354e+03   7.96e-01   9.07e+00   1.00e-01   5.83e+02s
 200  -1.9381e+05   1.11e-01   5.15e-01   1.00e-01   6.56e+02s
 400  -1.9678e+05   3.29e-02   1.88

In [ ]:
lambda_v = lambda_.value

#Забыл, что надо записывать w, поэтому на первой итерации у меня записаны lambda
w2 = w_count_and_record(lambda_v, y_1000_float64, X_1000_float64_1, inv_A, d)

np.save('all_data',lambda_v2)

In [12]:
print('The maximum memory consumption in the process = {0:0.3f} (Gb)\n'.format(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024 / 1024))

The maximum memory consumption in the process = 0.299 (Gb)

